# **MAIN CODE**

## **DATASET UPLOADING**

In [3]:
import os
from google.colab import drive
drive.mount('/content/drive')

dataset_path = "/content/drive/MyDrive/INTUTE_AI/MARBLE_CRACK_PROJECT/DeepCrack_Dataset"

folders = ["train_img", "train_lab", "test_img", "test_lab"]
for folder in folders:
    folder_path = os.path.join(dataset_path, folder)
    print(f"{folder}: {len(os.listdir(folder_path))} files found")


Mounted at /content/drive
train_img: 300 files found
train_lab: 300 files found
test_img: 237 files found
test_lab: 237 files found


## **TRAINING THE MODEL**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision import models
from PIL import Image
import os
import numpy as np

# Paths
data_dir = "/content/drive/MyDrive/INTUTE_AI/MARBLE_CRACK_PROJECT/DeepCrack"
train_img_dir = os.path.join(data_dir, "train_img")
train_lab_dir = os.path.join(data_dir, "train_lab")
test_img_dir = os.path.join(data_dir, "test_img")
test_lab_dir = os.path.join(data_dir, "test_lab")

# Hyperparameters
batch_size = 8
epochs = 20
learning_rate = 0.001
image_size = (256, 256)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Custom Dataset
class CrackDataset(Dataset):
    def __init__(self, img_dir, lab_dir, transform=None):
        self.img_dir = img_dir
        self.lab_dir = lab_dir
        self.img_list = sorted(os.listdir(img_dir))
        self.lab_list = sorted(os.listdir(lab_dir))
        self.transform = transform

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_list[idx])
        lab_path = os.path.join(self.lab_dir, self.lab_list[idx])

        img = Image.open(img_path).convert("RGB")
        label = Image.open(lab_path).convert("L")

        if self.transform:
            img = self.transform(img)
            label = self.transform(label)

        return img, label

# Transformations
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# Load Dataset
dataset = CrackDataset(train_img_dir, train_lab_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# U-Net Model
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        # Encoder (Downsampling)
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU()
        )
        self.pool1 = nn.MaxPool2d(2, 2)  # Reduces size to 128x128

        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU()
        )
        self.pool2 = nn.MaxPool2d(2, 2)  # Reduces size to 64x64

        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU()
        )
        self.pool3 = nn.MaxPool2d(2, 2)  # Reduces size to 32x32

        self.conv4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU()
        )
        self.pool4 = nn.MaxPool2d(2, 2)  # Reduces size to 16x16

        # Bottleneck
        self.conv5 = nn.Sequential(
            nn.Conv2d(512, 1024, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(1024, 1024, kernel_size=3, padding=1),
            nn.ReLU()
        )

        # Decoder (Upsampling)
        self.up6 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)  # Upsample to 32x32
        self.conv6 = nn.Sequential(
            nn.Conv2d(1024, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU()
        )

        self.up7 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)  # Upsample to 64x64
        self.conv7 = nn.Sequential(
            nn.Conv2d(512, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU()
        )

        self.up8 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)  # Upsample to 128x128
        self.conv8 = nn.Sequential(
            nn.Conv2d(256, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU()
        )

        self.up9 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)  # Upsample to 256x256
        self.conv9 = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU()
        )

        self.final = nn.Conv2d(64, 1, kernel_size=1)  # Output 1 channel

    def forward(self, x):
        c1 = self.conv1(x)
        p1 = self.pool1(c1)

        c2 = self.conv2(p1)
        p2 = self.pool2(c2)

        c3 = self.conv3(p2)
        p3 = self.pool3(c3)

        c4 = self.conv4(p3)
        p4 = self.pool4(c4)

        c5 = self.conv5(p4)

        up_6 = self.up6(c5)
        merge6 = torch.cat([up_6, c4], dim=1)
        c6 = self.conv6(merge6)

        up_7 = self.up7(c6)
        merge7 = torch.cat([up_7, c3], dim=1)
        c7 = self.conv7(merge7)

        up_8 = self.up8(c7)
        merge8 = torch.cat([up_8, c2], dim=1)
        c8 = self.conv8(merge8)

        up_9 = self.up9(c8)
        merge9 = torch.cat([up_9, c1], dim=1)
        c9 = self.conv9(merge9)

        output = torch.sigmoid(self.final(c9))  # Apply sigmoid to normalize output (0 to 1)
        return output


model = UNet().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training Loop
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss/len(dataloader):.4f}")

# Save Model
torch.save(model.state_dict(), "crack_segmentation.pth")
print("Training complete and model saved!")


Epoch [1/20], Loss: 0.1386
Epoch [2/20], Loss: 0.0628
Epoch [3/20], Loss: 0.0595
Epoch [4/20], Loss: 0.0517
Epoch [5/20], Loss: 0.0517
Epoch [6/20], Loss: 0.0507
Epoch [7/20], Loss: 0.0499
Epoch [8/20], Loss: 0.0492
Epoch [9/20], Loss: 0.0500
Epoch [10/20], Loss: 0.0474
Epoch [11/20], Loss: 0.0478
Epoch [12/20], Loss: 0.0486
Epoch [13/20], Loss: 0.0509
Epoch [14/20], Loss: 0.0453
Epoch [15/20], Loss: 0.0452
Epoch [16/20], Loss: 0.0467
Epoch [17/20], Loss: 0.0477
Epoch [18/20], Loss: 0.0453
Epoch [19/20], Loss: 0.0444
Epoch [20/20], Loss: 0.0444
Training complete and model saved!


## **TESTING THE MODEL**

In [ ]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Define paths
data_dir = "/content/drive/MyDrive/INTUTE_AI/MARBLE_CRACK_PROJECT/DeepCrack"
test_img_dir = os.path.join(data_dir, "test_img")
pred_output_dir = os.path.join(data_dir, "predicted_output")
os.makedirs(pred_output_dir, exist_ok=True)

# Load trained model
model = UNet()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load("/content/crack_segmentation.pth", map_location=device))
model.to(device)
model.eval()

# Define image transformation
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Process test images
for img_name in os.listdir(test_img_dir):
    img_path = os.path.join(test_img_dir, img_name)
    image = Image.open(img_path).convert("RGB")
    input_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)
        output_image = output.squeeze().cpu().numpy()

    # Normalize output to 0-255
    output_image = (output_image - output_image.min()) / (output_image.max() - output_image.min())
    output_image = (output_image * 255).astype(np.uint8)

    # Save predicted image
    output_img_path = os.path.join(pred_output_dir, img_name)
    Image.fromarray(output_image).save(output_img_path)
    print(f"Saved: {output_img_path}")

print("Processing complete!")


Saved: /content/drive/MyDrive/INTUTE_AI/MARBLE_CRACK_PROJECT/DeepCrack/predicted_output/IMG_6522-3.jpg
Saved: /content/drive/MyDrive/INTUTE_AI/MARBLE_CRACK_PROJECT/DeepCrack/predicted_output/IMG_6536-4.jpg
Saved: /content/drive/MyDrive/INTUTE_AI/MARBLE_CRACK_PROJECT/DeepCrack/predicted_output/IMG_6538-3.jpg
Saved: /content/drive/MyDrive/INTUTE_AI/MARBLE_CRACK_PROJECT/DeepCrack/predicted_output/IMG_6544-1.jpg
Saved: /content/drive/MyDrive/INTUTE_AI/MARBLE_CRACK_PROJECT/DeepCrack/predicted_output/IMG_6516-4.jpg
Saved: /content/drive/MyDrive/INTUTE_AI/MARBLE_CRACK_PROJECT/DeepCrack/predicted_output/IMG_6542-2.jpg
Saved: /content/drive/MyDrive/INTUTE_AI/MARBLE_CRACK_PROJECT/DeepCrack/predicted_output/IMG_6516-3.jpg
Saved: /content/drive/MyDrive/INTUTE_AI/MARBLE_CRACK_PROJECT/DeepCrack/predicted_output/IMG_6516-2.jpg
Saved: /content/drive/MyDrive/INTUTE_AI/MARBLE_CRACK_PROJECT/DeepCrack/predicted_output/IMG_6526-2.jpg
Saved: /content/drive/MyDrive/INTUTE_AI/MARBLE_CRACK_PROJECT/DeepCrack/pr

## **ACCURACY CHECK**

In [ ]:
import os

predicted_output_dir = "/content/drive/MyDrive/INTUTE_AI/MARBLE_CRACK_PROJECT/DeepCrack/predicted_output"
test_lab_dir = "/content/drive/MyDrive/INTUTE_AI/MARBLE_CRACK_PROJECT/DeepCrack/test_lab"

# Check if folders exist
print(f"Predicted Output Folder Exists: {os.path.exists(predicted_output_dir)}")
print(f"Test Lab Folder Exists: {os.path.exists(test_lab_dir)}")

# Check number of files
pred_files = os.listdir(predicted_output_dir)
test_lab_files = os.listdir(test_lab_dir)

print(f"Total Predicted Files: {len(pred_files)}")
print(f"Total Ground Truth Files: {len(test_lab_files)}")

# Print mismatched files
mismatched_files = [f for f in pred_files if f not in test_lab_files]
if mismatched_files:
    print(f"Mismatched Files (Missing in test_lab): {mismatched_files}")
else:
    print("✅ All predicted files have corresponding test_lab files.")


Predicted Output Folder Exists: True
Test Lab Folder Exists: True
Total Predicted Files: 237
Total Ground Truth Files: 237
Mismatched Files (Missing in test_lab): ['IMG_6522-3.jpg', 'IMG_6536-4.jpg', 'IMG_6538-3.jpg', 'IMG_6544-1.jpg', 'IMG_6516-4.jpg', 'IMG_6542-2.jpg', 'IMG_6516-3.jpg', 'IMG_6516-2.jpg', 'IMG_6526-2.jpg', 'IMG_6538-1.jpg', 'IMG_6542-4.jpg', 'IMG_6542-3.jpg', 'IMG_6522-1.jpg', 'IMG_6537-4.jpg', 'IMG_6526-3.jpg', 'IMG_6537-1.jpg', 'IMG_6533-3.jpg', 'IMG_6516-1.jpg', 'IMG_6544-3.jpg', 'IMG_6542-1.jpg', 'IMG_6526-4.jpg', 'IMG_6544-4.jpg', 'IMG_6536-1.jpg', 'IMG_6544-2.jpg', 'IMG_6536-7.jpg', 'IMG_6528-3.jpg', 'IMG_6536-3.jpg', 'IMG_6537-2.jpg', 'IMG_6536-2.jpg', 'IMG_6542-5.jpg', 'IMG_6536-5.jpg', 'IMG_6542-7.jpg', 'IMG_6526-1.jpg', 'IMG_6528-1.jpg', 'IMG_6537-3.jpg', 'IMG_6537-5.jpg', '11301-6.jpg', '11296-8.jpg', 'IMG_6472-4.jpg', '11308-1.jpg', '11301-3.jpg', '11296-7.jpg', 'IMG_6472-5.jpg', '11309-1.jpg', 'IMG_6469-2.jpg', '11296-5.jpg', 'IMG_6469-4.jpg', '11301-4.jp

### **RENAMING THE OUTPUT IMAGES TO MATCH**

In [ ]:
import os
import re

predicted_output_dir = "/content/drive/MyDrive/INTUTE_AI/MARBLE_CRACK_PROJECT/DeepCrack/predicted_output"
test_lab_dir = "/content/drive/MyDrive/INTUTE_AI/MARBLE_CRACK_PROJECT/DeepCrack/test_lab"

# Get filenames
pred_files = os.listdir(predicted_output_dir)
test_lab_files = os.listdir(test_lab_dir)

# Extract base filenames (without extra numbers)
def clean_filename(filename):
    return re.sub(r"-\d+", "", filename)  # Removes "-1", "-2", etc.

# Create a mapping of clean names
file_mapping = {}
for file in pred_files:
    cleaned_name = clean_filename(file)
    if cleaned_name in test_lab_files:
        file_mapping[file] = cleaned_name

# Rename files to match test_lab
for old_name, new_name in file_mapping.items():
    old_path = os.path.join(predicted_output_dir, old_name)
    new_path = os.path.join(predicted_output_dir, new_name)
    os.rename(old_path, new_path)

print("✅ Files Renamed! Re-run the accuracy check.")


✅ Files Renamed! Re-run the accuracy check.


### **CONVERTING THE OUTPUT IMAGES FROM JPG TO PNG**

In [ ]:

import os
predicted_output_dir = "/content/drive/MyDrive/INTUTE_AI/MARBLE_CRACK_PROJECT/DeepCrack/predicted_output"

for filename in os.listdir(predicted_output_dir):
    if filename.endswith(".jpg"):
        new_filename = filename.replace(".jpg", ".png")
        old_path = os.path.join(predicted_output_dir, filename)
        new_path = os.path.join(predicted_output_dir, new_filename)
        os.rename(old_path, new_path)
        print(f"Renamed: {filename} ➝ {new_filename}")

print("✅ All .jpg files renamed to .png")


✅ All .jpg files renamed to .png


### **COMPARING THE PREDICTED TO THE ACTUAL**

In [ ]:
import os
import numpy as np
import cv2
from sklearn.metrics import jaccard_score, f1_score

# Paths
pred_folder = "/content/drive/MyDrive/INTUTE_AI/MARBLE_CRACK_PROJECT/DeepCrack/predicted_output"
gt_folder = "/content/drive/MyDrive/INTUTE_AI/MARBLE_CRACK_PROJECT/DeepCrack/test_lab"

# Get predicted and ground truth filenames
pred_files = {os.path.splitext(f)[0]: f for f in os.listdir(pred_folder)}
gt_files = {os.path.splitext(f)[0]: f for f in os.listdir(gt_folder)}

# Ensure we evaluate only on matching filenames
common_files = set(pred_files.keys()) & set(gt_files.keys())

ious = []
dices = []

for file in common_files:
    pred_path = os.path.join(pred_folder, pred_files[file])
    gt_path = os.path.join(gt_folder, gt_files[file])

    # Load images
    pred_img = cv2.imread(pred_path, cv2.IMREAD_GRAYSCALE)
    gt_img = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)

    if pred_img is None or gt_img is None:
        print(f"Skipping {file}: Could not load images correctly.")
        continue

    # Threshold to binary
    _, pred_bin = cv2.threshold(pred_img, 127, 1, cv2.THRESH_BINARY)
    _, gt_bin = cv2.threshold(gt_img, 127, 1, cv2.THRESH_BINARY)

    # Flatten arrays
    pred_flat = pred_bin.flatten()
    gt_flat = gt_bin.flatten()

    # Compute IoU and Dice
    from skimage.transform import resize

# Resize predicted image to match ground truth dimensions
pred_img_resized = resize(pred_img, gt_img.shape, anti_aliasing=True)

# Convert to binary (thresholding)
pred_bin = (pred_img_resized > 0.5).astype(int)
gt_bin = (gt_img > 0.5).astype(int)

# Flatten
pred_flat = pred_bin.flatten()
gt_flat = gt_bin.flatten()

iou = jaccard_score(gt_flat, pred_flat, average='binary')
dice = f1_score(gt_flat, pred_flat, average='binary')

ious.append(iou)
dices.append(dice)


# Compute mean scores
mean_iou = np.mean(ious) if ious else float('nan')
mean_dice = np.mean(dices) if dices else float('nan')


print("Model Evaluation Results:")
print(f"Mean IoU Score: {mean_iou:.4f}")
print(f"Mean Dice Coefficient: {mean_dice:.4f}")


Model Evaluation Results:
Mean IoU Score: 0.4389
Mean Dice Coefficient: 0.6101


In [5]:
import os
import numpy as np
import cv2
from sklearn.metrics import jaccard_score, f1_score, precision_score, recall_score

# Paths
pred_folder = "/content/drive/MyDrive/INTUTE_AI/MARBLE_CRACK_PROJECT/DeepCrack_Dataset/predicted_output"
gt_folder = "/content/drive/MyDrive/INTUTE_AI/MARBLE_CRACK_PROJECT/DeepCrack_Dataset/test_lab"

# Get predicted and ground truth filenames
pred_files = {os.path.splitext(f)[0]: f for f in os.listdir(pred_folder)}
gt_files = {os.path.splitext(f)[0]: f for f in os.listdir(gt_folder)}

# Ensure we evaluate only on matching filenames
common_files = set(pred_files.keys()) & set(gt_files.keys())

ious = []
dices = []
precisions = []
recalls = []
f1_scores = []

for file in common_files:
    pred_path = os.path.join(pred_folder, pred_files[file])
    gt_path = os.path.join(gt_folder, gt_files[file])

    # Load images
    pred_img = cv2.imread(pred_path, cv2.IMREAD_GRAYSCALE)
    gt_img = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)

    if pred_img is None or gt_img is None:
        print(f"Skipping {file}: Could not load images correctly.")
        continue

    # Resize predicted image to match ground truth size
    pred_img = cv2.resize(pred_img, (gt_img.shape[1], gt_img.shape[0]), interpolation=cv2.INTER_NEAREST)

    # Threshold to binary
    _, pred_bin = cv2.threshold(pred_img, 127, 1, cv2.THRESH_BINARY)
    _, gt_bin = cv2.threshold(gt_img, 127, 1, cv2.THRESH_BINARY)

    # Flatten arrays
    pred_flat = pred_bin.flatten()
    gt_flat = gt_bin.flatten()

    # Compute metrics
    iou = jaccard_score(gt_flat, pred_flat, average='binary')
    dice = f1_score(gt_flat, pred_flat, average='binary')
    precision = precision_score(gt_flat, pred_flat, average='binary')
    recall = recall_score(gt_flat, pred_flat, average='binary')
    f1 = f1_score(gt_flat, pred_flat, average='binary')

    ious.append(iou)
    dices.append(dice)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

# Compute mean scores
mean_iou = np.mean(ious) if ious else float('nan')
mean_dice = np.mean(dices) if dices else float('nan')
mean_precision = np.mean(precisions) if precisions else float('nan')
mean_recall = np.mean(recalls) if recalls else float('nan')
mean_f1 = np.mean(f1_scores) if f1_scores else float('nan')

print("Model Evaluation Results:")
print(f"Mean IoU Score: {mean_iou:.4f}")
print(f"Mean Dice Coefficient: {mean_dice:.4f}")
print(f"Mean Precision: {mean_precision:.4f}")
print(f"Mean Recall: {mean_recall:.4f}")
print(f"Mean F1-score: {mean_f1:.4f}")

Model Evaluation Results:
Mean IoU Score: 0.5023
Mean Dice Coefficient: 0.6202
Mean Precision: 0.9084
Mean Recall: 0.5423
Mean F1-score: 0.6202
